<a href="https://colab.research.google.com/github/quent1fvr/Infection_pulmonaire/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0 - Importation des données et librairies**



In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow
import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import TimeDistributed
from keras.layers import Conv3D, BatchNormalization,MaxPooling3D, GlobalMaxPool3D
from keras.layers import TimeDistributed, GRU, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix

Pour l'import des datasets on crée une fonction dont les paramètres sont les chemins des fichiers hdf5 que l'on souhaite utiliser pour l'apprentissage.

Les fichiers hdf5 sont générés par le notebook "pre_processing.ipynb". 
Pour faire notre choix de pré-processing, il suffit de choisir les fichiers hdf5 dont les méthodes du notebook sont indiquées dans le nom du fichier : 
"dataset_..._methode_num1_num2"

In [52]:
def importation(path_normal, path_malade):
  hf_normal = h5py.File(path_normal, "r")
  hf_malade = h5py.File(path_malade, "r")

  # conversion des données en tableaux numpy
  Data_normal = np.array(hf_normal["dataset_1"][:])
  Data_malade = np.array(hf_malade["dataset_2"][:])

  return [Data_normal, Data_malade]

# **1 - Etude des données et création des labels**



In [46]:
Dataset = importation("/content/drive/MyDrive/UV PROJET P6/Dataset_normal.hdf5", 
                      "/content/drive/MyDrive/UV PROJET P6/Dataset_malade.hdf5")

In [47]:
Data_normal = Dataset[0]
Data_malade = Dataset[1]
Data_normal.shape, Data_malade.shape

((55720, 48, 48), (54600, 48, 48))

Afin de pouvoir utiliser des séquences dans les modèles suivant, nous modifions la shape des tableaux en créant des séquences de taille 70. 

In [48]:
Data_normal = Data_normal.reshape(Data_normal.shape[0]//70, 70, 48, 48)
Data_malade = Data_malade.reshape(Data_malade.shape[0]//70, 70, 48, 48)
Data = np.concatenate((Data_normal, Data_malade))
Data.shape

(1576, 70, 48, 48)

Le jeu de données est composé de séquences de 70 images par scan, c'est pourquoi on attribue un label pour chaque séquence :
- 0 : La personne est négative au COVID
- 1 : La personne est positive au COVID

In [49]:
Y_normal = np.zeros(Data_normal.shape[0], dtype=np.int8)
Y_malade = np.ones(Data_malade.shape[0], dtype=np.int8)
Y = np.concatenate((Y_normal, Y_malade))
Y.shape

(1576,)

Afin de faciliter le traitement pour la suite on encapsule la création des labels dans une fonction :

In [50]:
def labelling(Dataset):
  Data_normal = Dataset[0]
  Data_malade = Dataset[1]
  Data_normal = Data_normal.reshape(Data_normal.shape[0]//70, 70, 48, 48)
  Data_malade = Data_malade.reshape(Data_malade.shape[0]//70, 70, 48, 48)
  Data = np.concatenate((Data_normal, Data_malade))
  Y_normal = np.zeros(Data_normal.shape[0], dtype=np.int8)
  Y_malade = np.ones(Data_malade.shape[0], dtype=np.int8)
  Y = np.concatenate((Y_normal, Y_malade))
  return [Data, Y]

# **2 - Création des modèles d'apprentissage**




## Définition des modèles ▶ Choix à faire

### Modèle 1 : 2D

### Modèle 2 : 3DCNN




In [55]:
# création du modèle d'apprentissage

def build_convnet(shape=(70, 48, 48)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv3D(64, (3, 3, 3), input_shape=shape, padding='same', activation='relu'))
    model.add(Conv3D(64, (3, 3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPooling3D())
    
    model.add(Conv3D(128, (3, 3, 3), padding='same', activation='relu'))
    model.add(Conv3D(128, (3, 3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(GlobalMaxPool3D())
    return model

In [56]:
# création du réseau de décision

def action_model(shape=(70, 48, 48, 1), nbout=1):
    convnet = build_convnet(shape)
    
    model = keras.Sequential()
    model.add(convnet)
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='sigmoid'))
    return model

### Modèle 3 ...



## Définition des paramètres globaux

In [10]:
SIZE = (48, 48)
NBFRAME = 70
EPOCH = 10
BS = 8
CHANNEL = 1
INSHAPE = (70, 48, 48, 1)
model = action_model(INSHAPE, nbout=1)
optimizer = tensorflow.keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'binary_crossentropy',
    metrics=['acc']
)

On fixe la graine aléatoire afin que l'on puisse avoir des résultats similaires en relançant l'apprentissage : 

In [11]:
def fix_seed(seed):
    tensorflow.random.set_seed(seed)

SEED = 42
fix_seed(SEED)

On utilise les callbacks "EarlyStop" et "ModelCheckpoint" pour optimiser nos résultats : 

In [12]:
my_callbacks = [
    tensorflow.keras.callbacks.EarlyStopping(
        monitor = 'val_acc',
        min_delta = 0,
        patience = 10,
        verbose = 1,
        restore_best_weights = True),
    tensorflow.keras.callbacks.ModelCheckpoint(filepath='/tmp/myModel.h5')
]

## Définition des fonctions globales

#### Fonction de visualisation

In [23]:
def visualisation():
  acc = model.history.history['acc']
  val_acc = model.history.history['val_acc']
  loss = model.history.history['loss']
  val_loss = model.history.history['val_loss']

  print(acc)
  print(val_acc)

  print(loss)
  print(val_loss)

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

#### Fonction d'évaluation du modèle sur un jeu de données aléatoire

In [18]:
def evaluation(dataLength, iterations):
  
  for i in range(iterations):
    Data_test = []
    for i in range(dataLength):
      aleatoire = np.random.randint(0, Data_normal.shape[0])
      Data_test.append(Data_normal[aleatoire][:])
    for i in range(dataLength):
      aleatoire = np.random.randint(0, Data_malade.shape[0])
      Data_test.append(Data_malade[aleatoire][:])

    Data_test = np.array(Data_test)
    #Data_test.shape

    Y_test_normal = np.zeros(dataLength)
    Y_test_malade = np.ones(dataLength)
    Y_test = np.concatenate((Y_test_normal, Y_test_malade))
    model.evaluate(Data_test, Y_test)


In [ ]:
model.fit(Data, Y, epochs=EPOCH, validation_split=0.2, batch_size=BS, callbacks=my_callbacks)

On sauvegarde le modèle avec les meilleurs poids : 

In [19]:
model.load_weights("/tmp/myModel.h5")

In [ ]:
visualisation()

# **3 - Evaluation du modèle choisi**


## Modèle 1 : Conv2D

▶ run modèle conv2D

In [ ]:
my_callbacks = [
    tensorflow.keras.callbacks.EarlyStopping(
        monitor = 'val_acc',
        min_delta = 0,
        patience = 10,
        verbose = 1,
        restore_best_weights = True),
    tensorflow.keras.callbacks.ModelCheckpoint(filepath='/tmp/myModel1.h5')
]

In [ ]:
model.fit(Data, Y, epochs=EPOCH, validation_split=0.2, batch_size=BS, callbacks=my_callbacks)

On sauvegarde le modèle avec les meilleurs poids : 

In [ ]:
model.load_weights("/tmp/myModel1.h5")

In [ ]:
visualisation()

In [ ]:
evaluation(50, 10)

## Modèle 2 : 3DCNN

▶ run du modèle 3DCNN

In [ ]:
my_callbacks = [
    tensorflow.keras.callbacks.EarlyStopping(
        monitor = 'val_acc',
        min_delta = 0,
        patience = 10,
        verbose = 1,
        restore_best_weights = True),
    tensorflow.keras.callbacks.ModelCheckpoint(filepath='/tmp/myMode2.h5')
]

In [ ]:
model.fit(Data, Y, epochs=EPOCH, validation_split=0.2, batch_size=BS, callbacks=my_callbacks)

On sauvegarde le modèle avec les meilleurs poids : 

In [ ]:
model.load_weights("/tmp/myModel.h5")

In [ ]:
visualisation()

In [ ]:
evaluation(50, 10)

In [ ]:
# jspas ce que c'est
preds = np.round(model.predict(Data_test),0)
cm = confusion_matrix(Y_test, preds)
print(cm)

Le modèle est plus efficace que les modèles 2D, il prédit avec une accuracy de 93% sur un jeu de donnée aléatoire en test.

## Modèle 2 : 3DCNN + autre pre-processing

▶ run du modèle 3DCNN

In [ ]:
my_callbacks2 = [
    tensorflow.keras.callbacks.EarlyStopping(
        monitor = 'val_acc',
        min_delta = 0,
        patience = 10,
        verbose = 1,
        restore_best_weights = True),
    tensorflow.keras.callbacks.ModelCheckpoint(
        filepath='/tmp/myModel2.1.h5',    
        monitor="val_acc"))
]

On réinitialise les données :

In [54]:
Dataset = importation("/content/drive/MyDrive/UV PROJET P6/dataset_normal_methode_2.2_3.2.hdf5", 
                      "/content/drive/MyDrive/UV PROJET P6/dataset_malade_methode_2.2_3.2.hdf5")
Data = labelling(Dataset)[0]
Y = labelling(Dataset)[1]

In [ ]:
model.fit(Data, Y, epochs=EPOCH, validation_split=0.2, batch_size=BS, callbacks=my_callbacks2)

In [ ]:
model.load_weights("/tmp/myModel2.1.h5")

In [ ]:
visualisation()

In [ ]:
evaluation(50, 10)